In [ ]:
""" import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import nltk
from nltk.corpus import stopwords


# Load the data
df = pd.read_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\extracted_data.csv")

# Load pre-trained multilingual BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
model = BertModel.from_pretrained('bert-base-multilingual-uncased')

# Load Norwegian stopwords from NLTK
norwegian_stopwords = set(stopwords.words('norwegian'))

# Function to remove stopwords from text
def remove_stopwords(text):
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in norwegian_stopwords]
    return ' '.join(filtered_words)

# Function to generate embeddings for a given text
def generate_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

# Apply stopword removal to the content
df['content'] = df['content'].apply(remove_stopwords)

# Generate embeddings for each tender content
df['embeddings'] = df['content'].apply(lambda x: generate_embeddings(x))

# Function to handle search query and find the most relevant tenders
def recommend_tenders(query, df):
    query = remove_stopwords(query)  # Remove stopwords from the query as well
    query_embedding = generate_embeddings(query)
    
    # Compute similarity scores between the query and each document
    similarities = df['embeddings'].apply(lambda x: cosine_similarity([query_embedding], [x])[0][0])
    
    # Sort tenders based on similarity and return the top 5 recommendations
    top_recommendations = df.iloc[similarities.argsort()[-5:]]
    return top_recommendations[['filename', 'source_format', 'content']]

# Example usage
query = "cloud computing"
recommended_tenders = recommend_tenders(query, df)
print(recommended_tenders)
 """

In [ ]:
""" import gradio as gr
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity

# Load the data
df = pd.read_csv("C:/Users/spide/OneDrive/Desktop/Bachlorz/extracted_data.csv")

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to generate embeddings for a given text
def generate_embeddings(text):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embeddings.numpy()

# Generate embeddings for each tender content
df['embeddings'] = df['content'].apply(lambda x: generate_embeddings(x))

# Function to handle search query and find the most relevant tenders
def recommend_tenders(query, df):
    query_embedding = generate_embeddings(query)
    
    # Compute similarity scores between the query and each document
    similarities = df['embeddings'].apply(lambda x: cosine_similarity([query_embedding], [x])[0][0])
    
    # Sort tenders based on similarity and return the top 5 recommendations
    top_recommendations = df.iloc[similarities.argsort()[-5:]]
    
    # Add cosine similarity score to the result
    top_recommendations['cosine_similarity'] = similarities[similarities.argsort()[-5:]]
    
    return top_recommendations[['filename', 'source_format', 'content', 'cosine_similarity']]

# Gradio function for displaying recommendations
def recommend_tenders_gradio(query):
    recommended_tenders = recommend_tenders(query, df)
    return recommended_tenders

# Create Gradio Interface
interface = gr.Interface(fn=recommend_tenders_gradio, inputs="text", outputs="dataframe", title="Tender Recommendation System", description="Enter a search query to find the most relevant tenders.")

# Launch the interface
interface.launch()
 """

In [ ]:
""" # Gradio function for displaying recommendations
def recommend_tenders_gradio(query):
    recommended_tenders = recommend_tenders(query, df)
    return recommended_tenders

# Create Gradio Interface
interface = gr.Interface(fn=recommend_tenders_gradio, inputs="text", outputs="dataframe", live=True, title="Tender Recommendation System", description="Enter a search query to find the most relevant tenders.")

# Launch the interface
interface.launch() """


In [ ]:
""" query = "cloud computing krav" 
query_embedding = generate_embeddings(query)

# Compare similarity
similarity = cosine_similarity([query_embedding])
print("Cosine Similarity:", similarity[0][0])  # Expect a value between 0 and 1 """


In [2]:
import pandas as pd
import re
from transformers import BertModel, BertTokenizer
import torch

# Step 1: Load the CSV file into the DataFrame
df = pd.read_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\Data\extracted_data.csv")

# Step 2: Text Cleanup Function (removes unwanted characters, excessive spaces)
def clean_text(text):
    """
    This function cleans the extracted text by removing unwanted characters,
    excessive white spaces, and special characters.
    """
    # Remove excessive white spaces (replace multiple spaces with one)
    text = re.sub(r'\s+', ' ', text)  
    
    # Remove punctuation and special characters
    text = re.sub(r'[^\w\s]', '', text)  
    
    # Remove leading/trailing spaces
    text = text.strip()  
    
    return text

# Step 3: BERT Tokenizer and Model Setup (using a multilingual version of BERT)
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def get_bert_embeddings(text):
    """
    This function takes cleaned text as input, tokenizes it using BERT's tokenizer,
    and returns the contextual embeddings for the text.
    """
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Average pooling of token embeddings
    return embeddings

# Step 4: Clean the text in the 'content' column
df['cleaned_content'] = df['content'].apply(clean_text)

# Step 5: Generate BERT embeddings for the cleaned content
df['bert_embeddings'] = df['cleaned_content'].apply(get_bert_embeddings)

# Check the results
print(df[['content', 'cleaned_content', 'bert_embeddings']].head())

# Optional: Save the DataFrame with embeddings back to a new CSV
df.to_csv(r"C:\Users\spide\OneDrive\Desktop\Bachlorz\extracted_data_with_embeddings.csv", index=False)


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

c:\Users\spide\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\spide\.cache\huggingface\hub\models--bert-base-multilingual-cased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

                                             content  \
0  Nedre Romerike brann- og redningsvesen IKS Rev...   
1  Unnamed: 0 Unnamed: 1 Unnamed: 2 Unnamed: 3 Un...   
2  Vedlegg II.01 Bilag 2 - Seriøsitetskrav for by...   
3  MAL – Retningslinjer og krav til prøvedrift PR...   
4  KONKURRANSEGRUNNLAG ÅPEN ANBUDSKONKURRANSE ett...   

                                     cleaned_content  \
0  Nedre Romerike brann og redningsvesen IKS Revi...   
1  Unnamed 0 Unnamed 1 Unnamed 2 Unnamed 3 Unname...   
2  Vedlegg II01 Bilag 2  Seriøsitetskrav for bygg...   
3  MAL  Retningslinjer og krav til prøvedrift PRO...   
4  KONKURRANSEGRUNNLAG ÅPEN ANBUDSKONKURRANSE ett...   

                                     bert_embeddings  
0  [[tensor(-0.0473), tensor(-0.2503), tensor(0.4...  
1  [[tensor(0.2136), tensor(-0.0183), tensor(0.13...  
2  [[tensor(-0.2064), tensor(-0.4413), tensor(0.0...  
3  [[tensor(0.0175), tensor(-0.2266), tensor(0.55...  
4  [[tensor(0.1483), tensor(-0.2654), tensor(0.54..